<a href="https://colab.research.google.com/github/tada20001/NLP_2023/blob/main/CH18_04_TFBertForSequenceClassification_GPU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 19.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 20.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 74.9 MB/s eta 0:00:00


### 1. 데이터 로드 및 정제

In [2]:
import pandas as pd
import numpy as np
import urllib.request
import os
from tqdm import tqdm
import tensorflow as tf
from transformers import BertTokenizer, TFBertModel

In [3]:
urllib.request.urlretrieve("https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt", filename="ratings_train.txt")
urllib.request.urlretrieve("https://raw.githubusercontent.com/e9t/nsmc/master/ratings_test.txt", filename="ratings_test.txt")

('ratings_test.txt', <http.client.HTTPMessage at 0x7fe2cd19a560>)

In [4]:
train_data = pd.read_table('ratings_train.txt')
test_data = pd.read_table('ratings_test.txt')

In [5]:
train_data.shape, test_data.shape

((150000, 3), (50000, 3))

In [6]:
# 중복, Null 값 제거
train_data.drop_duplicates(subset=["document"], inplace=True)
train_data = train_data.dropna(how='any')
train_data = train_data.reset_index()
print('훈련데이터의 크기:', train_data.shape)

훈련데이터의 크기: (146182, 4)


In [7]:
test_data.drop_duplicates(subset=['document'], inplace=True)
test_data = test_data.dropna(how='any')
test_data = test_data.reset_index()
print('테스트 데이터의 크기 :', test_data.shape)

테스트 데이터의 크기 : (49157, 4)


### 2. 토크나이저를 이용한 정수 인코딩
* transformers에서 제공하는 BertTokenizerFast를 사용하여 klue-bert의 토크나이저를 로드함

In [8]:
from transformers import BertTokenizerFast
tokenizer = BertTokenizerFast.from_pretrained('klue/bert-base')

In [9]:
# 훈련데이터와 테스트 데이터로부터 리뷰데이터와 레이블을 별도로 분리하여 저장함
X_train_list = train_data['document'].tolist()
X_test_list = test_data['document'].tolist()
y_train = train_data['label'].tolist()
y_test = test_data['label'].tolist()

In [10]:
# 정수인코딩과 패딩을 수행함. 세그먼트 인코딩과 어텐션 마스크를 직접 만드는 것이 아니라 자동으로 만들어봄
X_train = tokenizer(X_train_list, truncation=True, padding=True)
X_test = tokenizer(X_test_list, truncation=True, padding=True)

In [11]:
# 데이터 확인
X_train[0]

Encoding(num_tokens=142, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing])

In [12]:
print(X_train[0].tokens)

['[CLS]', '아', '더', '##빙', '.', '.', '진짜', '짜증', '##나', '##네', '##요', '목소리', '[SEP]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD

In [13]:
print(X_train[0].ids)

[2, 1376, 831, 2604, 18, 18, 4229, 9801, 2075, 2203, 2182, 4243, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [14]:
print(X_train[0].type_ids)  # 세그먼트 인코딩.. 인식할 문장의 종류가 1개이므로 모두 0

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [15]:
print(X_train[0].attention_mask)  # padding 위치 알려

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


### 3. 데이터셋 생성 및 모델 학습

In [16]:
import tensorflow as tf
from transformers import TFBertForSequenceClassification
from tensorflow.keras.callbacks import EarlyStopping

In [17]:
# 데이터를 텐서플로우 데이터셋 형태로 변환
train_dataset = tf.data.Dataset.from_tensor_slices((
    dict(X_train), y_train
))

val_dataset = tf.data.Dataset.from_tensor_slices((
    dict(X_test), y_test
))

In [18]:
optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5)

transformers에서는 텍스트 분류를 위한 모델 구현체인 TFBertForSequenceClassification을 제공함

* TFBertForSequenceClassification.from_pretrained('모델 이름', num_labels=분류할 레이블의 수)



만약 위 코드를 사용하지 않으면, 앞서 진행한 방법에서 BERT의 마지막 레이어의 'CLS 토큰' 위치의 벡터를 꺼내와서 출력층으로 연결하는 일을 직접 해주어야 함

In [19]:
model = TFBertForSequenceClassification.from_pretrained('klue/bert-base', num_labels=2, from_pt=True)
model.compile(optimizer=optimizer, loss=model.hf_compute_loss, metrics=['accuracy'])

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertForSequenceClassification: ['bert.embeddings.position_ids']
- This IS expected if you are initializing TFBertForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


loss의 인자값으로 model.hf_compute_loss를 사용하였는데, 이는 TFBertForSequenceClassification의 hf_compute_loss는 다중 클래스 분류에서 사용하던 손실함수인 크로스 엔트로피함수가 맵핑되어져 있음



다음으로 다수의 에포크 학습을 사용할 때 조기종료를 위한 콜백 사용함. 여기에서는 min_delta를 0.001으로 설정했는데, 정확도 개선폭의 최소값을 설정해 두는 것임. 최소값 미만을 경우에는 개선되지 않았다고 보는 것임

In [20]:
early_stopping = EarlyStopping(
    monitor='val_accuracy',
    min_delta=0.001,
    patience=2
)

model.fit(train_dataset.shuffle(10000).batch(32), epochs=5, batch_size=32, 
          validation_data=val_dataset.shuffle(10000).batch(32), callbacks=[early_stopping])

Epoch 1/5
4569/4569 [==============================] - 4665s 1s/step - loss: 0.2816 - accuracy: 0.8809 - val_loss: 0.2572 - val_accuracy: 0.8927
Epoch 2/5
4569/4569 [==============================] - 4610s 1s/step - loss: 0.1957 - accuracy: 0.9226 - val_loss: 0.2714 - val_accuracy: 0.8967
Epoch 3/5
4569/4569 [==============================] - 4609s 1s/step - loss: 0.1327 - accuracy: 0.9502 - val_loss: 0.3212 - val_accuracy: 0.8936
Epoch 4/5
4569/4569 [==============================] - 4601s 1s/step - loss: 0.0952 - accuracy: 0.9657 - val_loss: 0.3797 - val_accuracy: 0.8936


In [21]:
!mk nsmc_model

/bin/bash: mk: command not found


In [22]:
# 모델 저장
model.save_pretrained('nsmc_model/bert-base')
tokenizer.save_pretrained('nsmc_model/bert-base')

('nsmc_model/bert-base/tokenizer_config.json',
 'nsmc_model/bert-base/special_tokens_map.json',
 'nsmc_model/bert-base/vocab.txt',
 'nsmc_model/bert-base/added_tokens.json',
 'nsmc_model/bert-base/tokenizer.json')

### 4. 테스트

* TextClassificationPipeline은 모델의 예측값이 소프트맥스 함수를 통과한 후의 값인 레이블별 스코어를 보여줌

In [23]:
from transformers import TextClassificationPipeline

loaded_tokenizer = BertTokenizerFast.from_pretrained('nsmc_model/bert-base')
loaded_model = TFBertForSequenceClassification.from_pretrained('nsmc_model/bert-base')

Some layers from the model checkpoint at nsmc_model/bert-base were not used when initializing TFBertForSequenceClassification: ['dropout_37']
- This IS expected if you are initializing TFBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertForSequenceClassification were initialized from the model checkpoint at nsmc_model/bert-base.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForSequenceClassification for predictions without further training.


In [26]:
# 임의의 문장에 대해 예측결과와 스코어 출력
# 기존 레이블이 0이었던 임의의 테스트 데이터를 검증
text_classifier = TextClassificationPipeline(
    tokenizer=loaded_tokenizer,
    model=loaded_model,
    framework='tf',
    return_all_scores=True
)

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/text_classification.py:104: UserWarning: `return_all_scores` is now deprecated,  if want a similar funcionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(


In [27]:
text_classifier('뭐야 이 평점들은.... 나쁘진 않지만 10점짜리는 더더욱 아니잖아')[0]

[{'label': 'LABEL_0', 'score': 0.9964809417724609},
 {'label': 'LABEL_1', 'score': 0.003519026329740882}]